In [7]:
import sys
sys.path.append('C:\\Users\\DavidB\\PycharmProjects\\My_Football_Analytics\\Basics')

import pandas as pd
import numpy as np
import Data.Metrica_IO as mio
from Tracking_Data import tracking_data
from mplsoccer import Pitch
import matplotlib.pyplot as plt
from floodlight.io import dfl
from lxml import etree


In [4]:
pos_filepath = 'DFL_04_02_positions_raw_DFL-COM-000001_DFL-MAT-0027AD.xml'
mi_filepath = 'DFL_02_01_matchinformation_DFL-COM-000001_DFL-MAT-0027AD.xml'

xy_objects, possession_objects, ballstatus_objects, teamsheets, pitch = dfl.read_position_data_xml(filepath_positions=pos_filepath, filepath_mat_info=mi_filepath)


In [5]:
xy_objects

{'firstHalf': {'Home': XY(xy=array([[ -8.23, -15.61, -16.54, ...,  18.66,    nan,    nan],
         [ -8.25, -15.63, -16.51, ...,  18.66,    nan,    nan],
         [ -8.26, -15.65, -16.49, ...,  18.66,    nan,    nan],
         ...,
         [ 18.38,  15.  ,  -7.03, ...,  28.76,    nan,    nan],
         [ 18.37,  14.94,  -7.01, ...,  28.69,    nan,    nan],
         [ 18.35,  14.89,  -7.  , ...,  28.62,    nan,    nan]]), framerate=25, direction=None),
  'Away': XY(xy=array([[  2.05, -33.52,    nan, ...,  25.26,    nan,    nan],
         [  2.01, -33.51,    nan, ...,  25.25,    nan,    nan],
         [  1.99, -33.51,    nan, ...,  25.24,    nan,    nan],
         ...,
         [ 21.78,  -1.72,    nan, ...,  27.83,    nan,    nan],
         [ 21.74,  -1.79,    nan, ...,  27.8 ,    nan,    nan],
         [ 21.71,  -1.85,    nan, ...,  27.76,    nan,    nan]]), framerate=25, direction=None),
  'Ball': XY(xy=array([[-0.38, -0.28],
         [-0.35, -0.3 ],
         [-0.33, -0.32],
        

In [35]:
teamsheets

{'Home': Teamsheet(teamsheet=              player position                 team  jID             pID  \
 0          M. Frantz       RM  Sport-Club Freiburg    8  DFL-OBJ-0000PC   
 1         Ç. Söyüncü      IVL  Sport-Club Freiburg    4  DFL-OBJ-002709   
 2        J. Schuster     None  Sport-Club Freiburg   23  DFL-OBJ-000013   
 3         A. Abrashi      DML  Sport-Club Freiburg    6  DFL-OBJ-00266D   
 4   F. Niederlechner      STL  Sport-Club Freiburg    7  DFL-OBJ-0002GE   
 5         P. Stenzel       RV  Sport-Club Freiburg   15  DFL-OBJ-0026MA   
 6      J. Föhrenbach       LV  Sport-Club Freiburg   25  DFL-OBJ-00266E   
 7        A. Schwolow       TW  Sport-Club Freiburg    1  DFL-OBJ-00001D   
 8           O. Bulut     None  Sport-Club Freiburg   11  DFL-OBJ-000195   
 9        N. Petersen      STR  Sport-Club Freiburg   18  DFL-OBJ-0000IU   
 10    G. Niedermeier     None  Sport-Club Freiburg   24  DFL-OBJ-0000LJ   
 11        M. Philipp      STR  Sport-Club Freiburg   26  DF

In [31]:
links_jID_to_xID = {
        "Home": teamsheets['Home'].get_links("jID", "xID"),
        "Away": teamsheets['Away'].get_links("jID", "xID"),
    }
links_pID_to_jID = {
        "Home": teamsheets['Home'].get_links("pID", "jID"),
        "Away": teamsheets['Away'].get_links("pID", "jID"),
    }

In [33]:
links_pID_to_jID

{'Home': {'DFL-OBJ-0000PC': 8,
  'DFL-OBJ-002709': 4,
  'DFL-OBJ-000013': 23,
  'DFL-OBJ-00266D': 6,
  'DFL-OBJ-0002GE': 7,
  'DFL-OBJ-0026MA': 15,
  'DFL-OBJ-00266E': 25,
  'DFL-OBJ-00001D': 1,
  'DFL-OBJ-000195': 11,
  'DFL-OBJ-0000IU': 18,
  'DFL-OBJ-0000LJ': 24,
  'DFL-OBJ-0002B9': 26,
  'DFL-OBJ-0001E6': 32,
  'DFL-OBJ-0002DW': 19,
  'DFL-OBJ-0000D9': 27,
  'DFL-OBJ-0001D5': 3,
  'DFL-OBJ-00017V': 30,
  'DFL-OBJ-0002EQ': 44},
 'Away': {'DFL-OBJ-0001V4': 13,
  'DFL-OBJ-0027IH': 41,
  'DFL-OBJ-0000U1': 1,
  'DFL-OBJ-0000U9': 4,
  'DFL-OBJ-0000IT': 33,
  'DFL-OBJ-0027BQ': 17,
  'DFL-OBJ-0000KF': 5,
  'DFL-OBJ-0000Y4': 27,
  'DFL-OBJ-00028Y': 30,
  'DFL-OBJ-00000W': 7,
  'DFL-OBJ-00270E': 25,
  'DFL-OBJ-0000UD': 15,
  'DFL-OBJ-0026T9': 21,
  'DFL-OBJ-0000IB': 22,
  'DFL-OBJ-0000VY': 28,
  'DFL-OBJ-0000ZC': 34,
  'DFL-OBJ-00000C': 14,
  'DFL-OBJ-0000P6': 11}}

In [19]:
data = dict()
for _, frame_set in etree.iterparse(pos_filepath, tag="FrameSet"):
    # get ball position
    if frame_set.get("TeamId").lower() == "ball":
        print(frame_set)
        segment = frame_set.get("GameSection")
        print(segment)
        data['ball'] = dict()
        data['ball']['x'] = []
        data['ball']['y'] = []
        for frame in frame_set:
            data['ball']['x'].append(frame.get('X'))            
            data['ball']['y'].append(frame.get('Y'))
            

<Element FrameSet at 0x2732909a680>
firstHalf
<Element FrameSet at 0x2732943d600>
secondHalf


In [251]:
data = dict()
data['ball'] = dict()
data['ball']['x'] = []
data['ball']['y'] = []
data['ballstatus'] = []
data['possession'] = []
data['GameSection'] = []
data['Time'] = []
firsthalf_time = datetime.datetime(100,1,1, 0, 0, 0, 0)
secondhalf_time = datetime.datetime(100,1,1, 0, 45, 0, 0)

periods, est_framerate = dfl._create_periods_from_dat(pos_filepath)

for _, frame_set in etree.iterparse(pos_filepath, tag="FrameSet"):
    # get ball position
    print(frame_set)
    segment = frame_set.get("GameSection")
    print(segment)
    # get ball position
    if frame_set.get("TeamId").lower() == "ball":
        for frame in frame_set:
            data['ball']['x'].append(frame.get('X'))            
            data['ball']['y'].append(frame.get('Y'))
            data['ballstatus'].append(float(frame.get("BallStatus")))
            data['possession'].append(float(frame.get("BallPossession")))
            data['GameSection'].append(frame_set.get("GameSection"))
            if segment == 'firstHalf':
                data['Time'].append(firsthalf_time.time().strftime('%H:%M:%S.%f'))
                firsthalf_time = firsthalf_time + datetime.timedelta(milliseconds=40)
            elif segment == 'secondHalf':
                data['Time'].append(secondhalf_time.time().strftime('%H:%M:%S.%f'))
                secondhalf_time = secondhalf_time + datetime.timedelta(milliseconds=40)
    else:
        frames = [frame for frame in frame_set.iterfind("Frame")]
        print(frame_set.get('PersonId'))
        print(int(frames[0].get('N')))
        if int(frames[0].get('N')) >= periods['firstHalf'][0] and int(frames[0].get('N')) <= periods['firstHalf'][1]:
            start = int(frames[0].get('N')) - periods['firstHalf'][0]            
            print(start)
        elif int(frames[0].get('N')) >= periods['secondHalf'][0] and int(frames[0].get('N')) <= periods['secondHalf'][1]:
            start = int(frames[0].get('N')) - periods['secondHalf'][0] + periods['firstHalf'][1]
            print(start)
        if frame_set.get("PersonId") in links_pID_to_jID["Home"]:
            jrsy = links_pID_to_jID['Home'][frame_set.get("PersonId")]
            # only for the first time we need to intitalize, then it already exists (for second half)
            if f'Home{jrsy}' not in data.keys():
                data[f'Home{jrsy}'] = dict()
                data[f'Home{jrsy}']['x'] = list(np.repeat(np.nan, start))
                data[f'Home{jrsy}']['y'] = list(np.repeat(np.nan, start))
            for frame in frames:
                data[f'Home{jrsy}']['x'].append(frame.get("X"))
                data[f'Home{jrsy}']['y'].append(frame.get("Y"))
        elif frame_set.get("PersonId") in links_pID_to_jID["Away"]:
            jrsy = links_pID_to_jID['Away'][frame_set.get("PersonId")]
            if f'Away{jrsy}' not in data.keys():
                data[f'Away{jrsy}'] = dict()
                data[f'Away{jrsy}']['x'] = list(np.repeat(np.nan, start))
                data[f'Away{jrsy}']['y'] = list(np.repeat(np.nan, start))
            for frame in frames:
                data[f'Away{jrsy}']['x'].append(frame.get("X"))
                data[f'Away{jrsy}']['y'].append(frame.get("Y"))



<Element FrameSet at 0x27c78d34ac0>
firstHalf
DFL-OBJ-00017V
10001
0
<Element FrameSet at 0x27c78d34c40>
firstHalf
DFL-OBJ-0000D9
10001
0
<Element FrameSet at 0x27c74715e80>
firstHalf
<Element FrameSet at 0x27c31795000>
firstHalf
DFL-OBJ-0000PC
10001
0
<Element FrameSet at 0x27c74714880>
firstHalf
DFL-OBJ-0026MA
10001
0
<Element FrameSet at 0x27c3e9b9380>
firstHalf
DFL-OBJ-0001E6
10001
0
<Element FrameSet at 0x27c3e92d880>
firstHalf
DFL-OBJ-00000W
10001
0
<Element FrameSet at 0x27c65240f00>
firstHalf
DFL-OBJ-0000Y4
10001
0
<Element FrameSet at 0x27c312a4d40>
firstHalf
DFL-OBJ-00266D
10001
0
<Element FrameSet at 0x27c74714080>
firstHalf
DFL-OBJ-00028Y
10001
0
<Element FrameSet at 0x27c746faa00>
firstHalf
DFL-OBJ-0000ZC
10001
0
<Element FrameSet at 0x27c33957bc0>
firstHalf
DFL-OBJ-0000IB
10001
0
<Element FrameSet at 0x27c746afec0>
firstHalf
DFL-OBJ-002709
10001
0
<Element FrameSet at 0x27c75ecd780>
firstHalf
DFL-OBJ-0001V4
10001
0
<Element FrameSet at 0x27c3e52bf00>
firstHalf
DFL-OBJ-000

In [254]:
for key in data.keys():
    if len(data[key]) == 2:
        for k2 in data[key]:
            print(key, k2)
            print(len(data[key][k2]))
    else:
        print(key)
        print(len(data[key]))

ball x
137219
ball y
137219
ballstatus
137219
possession
137219
GameSection
137219
Time
137219
Home30 x
127622
Home30 y
127622
Home27 x
133156
Home27 y
133156
Home8 x
137219
Home8 y
137219
Home15 x
137219
Home15 y
137219
Home32 x
137219
Home32 y
137219
Away7 x
131913
Away7 y
131913
Away27 x
137219
Away27 y
137219
Home6 x
86698
Home6 y
86698
Away30 x
137219
Away30 y
137219
Away34 x
137219
Away34 y
137219
Away22 x
137219
Away22 y
137219
Home4 x
137219
Home4 y
137219
Away13 x
137219
Away13 y
137219
Home7 x
130023
Home7 y
130023
Away1 x
137219
Away1 y
137219
Away14 x
111741
Away14 y
111741
Away33 x
122324
Away33 y
122324
Home1 x
137219
Home1 y
137219
Home26 x
137219
Home26 y
137219
Home3 x
137219
Home3 y
137219
Away15 x
137219
Away15 y
137219
Away5 x
137219
Away5 y
137219
Away4 x
147219
Away4 y
147219
Home18 x
147219
Home18 y
147219
Away11 x
147219
Away11 y
147219
Away17 x
147219
Away17 y
147219
Home19 x
147219
Home19 y
147219
Home25 x
147219
Home25 y
147219


In [259]:
data['Home25']['x']

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [214]:
import flatdict
flat_data = flatdict.FlatDict(data, delimiter='_')
flat_data.keys()
df = pd.DataFrame.from_dict(flat_data, orient = 'index').transpose()
df

,ball_x,ball_y,ballstatus,possession,GameSection,Time,Home30_x,Home30_y,Home27_x,Home27_y,...,Home18_x,Home18_y,Away11_x,Away11_y,Away17_x,Away17_y,Home19_x,Home19_y,Home25_x,Home25_y
0,-0.38,-0.28,1.0,2.0,firstHalf,00:00:00.000000,-15.89,18.66,-9.70,-5.98,...,-1.22,-33.30,-1.64,-33.06,0.14,-33.27,0.85,-29.02,-0.24,-32.35
1,-0.35,-0.30,1.0,2.0,firstHalf,00:00:00.040000,-15.89,18.66,-9.71,-5.99,...,-1.24,-33.23,-1.62,-33.07,0.13,-33.29,0.85,-29.01,-0.24,-32.35
2,-0.33,-0.32,1.0,2.0,firstHalf,00:00:00.080000,-15.89,18.66,-9.73,-6.00,...,-1.26,-33.15,-1.60,-33.09,0.12,-33.31,0.85,-29.00,-0.24,-32.35
3,-0.31,-0.33,1.0,2.0,firstHalf,00:00:00.120000,-15.89,18.66,-9.75,-6.02,...,-1.27,-33.06,-1.58,-33.10,0.11,-33.33,0.85,-28.98,-0.24,-32.35
4,1.09,-0.13,1.0,2.0,firstHalf,00:00:00.160000,-15.88,18.66,-9.77,-6.03,...,-1.29,-32.96,-1.56,-33.11,0.10,-33.35,0.85,-28.95,-0.24,-32.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137214,23.04,6.63,0.0,2.0,secondHalf,01:31:17.360000,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
137215,23.02,6.62,0.0,2.0,secondHalf,01:31:17.400000,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
137216,23.01,6.62,0.0,2.0,secondHalf,01:31:17.440000,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
137217,23.00,6.61,0.0,2.0,secondHalf,01:31:17.480000,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [222]:
pd.DataFrame.from_dict(flat_data, orient = 'index')

,0,1,2,3,4,5,6,7,8,9,...,137209,137210,137211,137212,137213,137214,137215,137216,137217,137218
ball_x,-0.38,-0.35,-0.33,-0.31,1.09,1.57,2.02,2.49,2.94,3.35,...,23.08,23.08,23.07,23.06,23.04,23.04,23.02,23.01,23.00,22.98
ball_y,-0.28,-0.30,-0.32,-0.33,-0.13,-0.32,-0.48,-0.70,-0.92,-1.14,...,6.64,6.64,6.64,6.64,6.63,6.63,6.62,6.62,6.61,6.60
ballstatus,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
possession,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
GameSection,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,firstHalf,...,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf,secondHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Away17_y,-33.27,-33.29,-33.31,-33.33,-33.35,-33.37,-33.40,-33.41,-33.44,-33.46,...,None,None,None,None,None,None,None,None,None,None
Home19_x,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,0.85,...,None,None,None,None,None,None,None,None,None,None
Home19_y,-29.02,-29.01,-29.00,-28.98,-28.95,-28.93,-28.89,-28.86,-28.80,-28.75,...,None,None,None,None,None,None,None,None,None,None
Home25_x,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,-0.24,...,None,None,None,None,None,None,None,None,None,None


In [223]:
len(data['ball']['y'])

137219

In [225]:
len(flat_data['Home30_x'])

127622

In [230]:
for key in data.keys():
    if len(data[key]) == 2:
        for k2 in data[key]:
            print(key, k2)
            print(len(data[key][k2]))
    else:
        print(key)
        print(len(data[key]))

ball x
137219
ball y
137219
ballstatus
137219
possession
137219
GameSection
137219
Time
137219
Home30 x
127622
Home30 y
127622
Home27 x
133156
Home27 y
133156
Home8 x
137219
Home8 y
137219
Home15 x
137219
Home15 y
137219
Home32 x
137219
Home32 y
137219
Away7 x
131913
Away7 y
131913
Away27 x
137219
Away27 y
137219
Home6 x
86698
Home6 y
86698
Away30 x
137219
Away30 y
137219
Away34 x
137219
Away34 y
137219
Away22 x
137219
Away22 y
137219
Home4 x
137219
Home4 y
137219
Away13 x
137219
Away13 y
137219
Home7 x
130023
Home7 y
130023
Away1 x
137219
Away1 y
137219
Away14 x
111741
Away14 y
111741
Away33 x
122324
Away33 y
122324
Home1 x
137219
Home1 y
137219
Home26 x
137219
Home26 y
137219
Home3 x
137219
Home3 y
137219
Away15 x
137219
Away15 y
137219
Away5 x
137219
Away5 y
137219
Away4 x
5306
Away4 y
5306
Home18 x
50521
Home18 y
50521
Away11 x
25478
Away11 y
25478
Away17 x
14895
Away17 y
14895
Home19 x
7196
Home19 y
7196
Home25 x
4063
Home25 y
4063


In [240]:
periods, est_framerate = dfl._create_periods_from_dat(pos_filepath)
periods

{'firstHalf': (10001, 77780), 'secondHalf': (100001, 169439)}